In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sympy import diff, symbols, Symbol, integrate, solve, sqrt, log, sin, cos, lambdify
%matplotlib inline

# Задание 1. Линейные системы. Устойчивость численных методов.

Решить численно задачу о колебаниях в системе, где и возвращающая сила, и
коэффициент вязкого трения убывают со временем (уравнение Эйлера):
$$
\ddot{x} + 100\,\dfrac{x}{t^2} = 0
$$ 
Сначала решим численно!
Подстановка $x = t^\alpha$ дает следующее уравнение:
$$
\boxed{
    x (t) = C_1 \sqrt{t} \sin \left( \frac{1}{2} \sqrt{399} \ln{t} \right) + C_2 \sqrt{t} \cos \left( \frac{1}{2} \sqrt{399} \ln{t} \right)
}
$$

In [2]:
# Запишем решение
C1, C2, t = symbols('C1 C2 t')
sol = C1*sqrt(t)*sin(0.5*sqrt(399)*log(t)) \
                        + C2*sqrt(t)*cos(0.5*sqrt(399)*log(t))
sol_d1 = lambdify(t, diff(sol, t))
C1_sol, C2_sol = list(solve([sol_d1(1) - 1, sol.subs(t, 1) - 1]).values())
t_theor = lambdify(t, sol.subs(C1, C1_sol).subs(C2, C2_sol))

In [3]:
def compare_results(theor, predicted):
    print('  Вычисленное x(101) = ' + str(predicted))
    print('  Теоретическое x(101) = ' + str(theor))

In [4]:
system_of_eq = lambda t, x: np.array([
    1.0,  # Время
    x[2],  # x точка
    -100.0*x[1]/x[0]**2,  # p точка
])

In [5]:
x_start = np.array([1.0, 1.0, 1.0])
t_stop = 101.0
h = 0.1

## Методы Эйлера

Для решения методом Эйлера разобьем на нормальную систему уравнений:

$$
\begin{cases}
    \dot{x} = p \\
    \dot{p} = -100\, \cfrac{x}{t^2}
\end{cases}
$$

### Явный метод Эйлера

In [6]:
# Явный метод Эйлера
def eiler_1(funcs, x_start, t_stop, h=0.1):
    # Начальное значение
    x = x_start
    while x[0] < t_stop:
        x += h*funcs(x)
    return x

In [7]:
# Параметр времени не используется, но мы передаем его для правильной работы scipy
x_1 = eiler_1(lambda x: system_of_eq(0, x), x_start.copy(), t_stop, h)

In [8]:
compare_results(t_theor(t_stop), x_1[1])

  Вычисленное x(101) = 387.60543228423273
  Теоретическое x(101) = -4.73890168470781


Результаты очень плохие. Это объясняется тем, что явный метод Эйлера дает ошибку $o (h)$, которая в нашем случае оказалась велика.

### Неявный метод Эйлера

In [9]:
# Неявный метод Эйлера
def eiler_2(funcs, x_start, t_stop, h=0.1):
    # Начальное значение
    x = x_start
    while x[0] < t_stop:
        # Прогноз
        x_tmp_tmp = x + h*funcs(x)
        # Пересчет
        x_tmp = x + h*(funcs(x) + funcs(x_tmp_tmp))/2
        x += h*(funcs(x) + funcs(x_tmp))/2
    return x

In [10]:
# Параметр времени не используется, но мы передаем его для правильной работы scipy
x_2 = eiler_2(lambda x: system_of_eq(0, x), x_start.copy(), t_stop, h)

In [11]:
compare_results(t_theor(t_stop), x_2[1])

  Вычисленное x(101) = -1.7472631764270972
  Теоретическое x(101) = -4.73890168470781


Неявный метод Эйлера дает неплохие результаты. Более того, он оказывается самым точным, поскольку дает точность $o(h^2)$, к тому же, имеет повышенную устройчивость и позволяет обойти некоторые проблемы при решении жестких систем ДУ.

### Метод Эйлера с центральной точкой

In [12]:
# метод Эйлера с центральной точкой
def eiler_3(funcs, x_start, t_stop, h=0.1):
    # Начальное значение
    x = x_start
    while x[0] < t_stop:
        # Нам придется "подгядеть" f_(i+1), а затем взять полусумму с f_i
        x_tmp = (funcs(x + h*funcs(x)) + funcs(x))/2
        # Результат прибавлять в качестве вектора касательной
        x += h*x_tmp
    return x

In [13]:
# Параметр времени не используется, но мы передаем его для правильной работы scipy
x_3 = eiler_3(lambda x: system_of_eq(0, x), x_start.copy(), t_stop, h)

In [14]:
compare_results(t_theor(t_stop), x_3[1])

  Вычисленное x(101) = -13.961618490145836
  Теоретическое x(101) = -4.73890168470781


Метод Эйлера с центральной точкой дает результаты лучше явного, но хуже неявного (из-за меньшей устойчивости по сравнению с неявным).

## Метод Дормана-Принса

In [15]:
from scipy.integrate import solve_ivp

In [16]:
sol = solve_ivp(system_of_eq, (x_start[0], t_stop), x_start, method='RK45')

In [17]:
compare_results(t_theor(t_stop), sol.y[1][-1])

  Вычисленное x(101) = -4.820773818864743
  Теоретическое x(101) = -4.73890168470781


Метод Дормана-Принса дает хорошие результаты интегрирования.